In [3]:
from fontTools.misc.cython import returns
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

from caen_felib import device, lib
import pickle

# Get CAEN FELib path
print(f'CAEN FELib found at: {lib.path} (version={lib.version})')


def save_waveforms(event_num, timestamp, sampling_rate, time_resolution, waveforms):
	with open('raw.bin', 'ab') as f:
		f.write(struct.pack('I', event_num))
		f.write(struct.pack('Q', timestamp))
		f.write(struct.pack('I', sampling_rate))
		f.write(struct.pack('Q', time_resolution))

		for waveform in waveforms:
			for point in waveform:
				f.write(struct.pack('f', point))
                
                
def startACQ(n):
	dig.cmd.ArmAcquisition()
	dig.cmd.SwStartAcquisition()
	
	event_nums = []
	waveforms = []
	timestamps = []
	waveform_sizes = []

	for i in range(n):
		dig.cmd.SendSwTrigger()
		dig.endpoint.scope.read_data(-1, data)
		waveforms.append(waveform)
		timestamps.append(timestamp)
		waveform_sizes.append(waveform_size)

	dig.cmd.DisarmAcquisition()
    return waveforms, timestamps, waveform_sizes

CAEN FELib found at: libCAEN_FELib.so (version=1.3.1)


In [123]:
dig = device.connect('dig2://caen.internal/usb/51054')

print(f'Connected to digitizer (handle={hex(dig.handle)}, name={dig.name})')

Connected to digitizer (handle=0xcae0000000000000, name=)


In [124]:
n_ch = int(dig.par.NUMCH.value)
adc_samplrate_msps = float(dig.par.ADC_SAMPLRATE.value)  # in Msps
adc_n_bits = int(dig.par.ADC_NBIT.value)
sampling_period_ns = int(1e3 / adc_samplrate_msps)
fw_type = dig.par.FWTYPE.value
print(f'Number of channels: {n_ch}, ADC sampling rate: {adc_samplrate_msps} Msps, ADC resolution: {adc_n_bits} bits, sampling period: {sampling_period_ns} ns, firmware type: {fw_type}')

Number of channels: 64, ADC sampling rate: 125.0 Msps, ADC resolution: 16 bits, sampling period: 8 ns, firmware type: Scope


In [125]:
# Configuration parameters
reclen_ns = 300000  # in ns
pretrg_ns = 16336  # in ns

# Configure digitizer
dig.par.RECORDLENGTHT.value = f'{reclen_ns}'
dig.par.PRETRIGGERT.value = f'{pretrg_ns}'
dig.par.AcqTriggerSource.value = 'SwTrg'


for ch in dig.ch:
	ch.par.DCOffset.value = f'50'

data_format = [
	{
		'name': 'TIMESTAMP',
		'type': 'U64',
	},
	{
		'name': 'WAVEFORM',
		'type': 'U16',
		'dim': 2,
		'shape': [n_ch, reclen_ns],
	},
	{
		'name': 'WAVEFORM_SIZE',
		'type': 'U64',
		'dim': 1,
		'shape': [n_ch],
	},
]

data = dig.endpoint.scope.set_read_data_format(data_format)
timestamp = data[0].value
waveform = data[1].value
waveform_size = data[2].value

dig.endpoint.par.ActiveEndpoint.value = 'scope'

In [130]:
startACQ(100)